In [ ]:
import torch
from transformers import SamModel, SamProcessor

device = "cuda" if torch.cuda.is_available() else "cpu"

processor = SamProcessor.from_pretrained("facebook/sam-vit-huge")
model = SamModel.from_pretrained("facebook/sam-vit-huge").to(device)

In [ ]:

def get_bounding_box(ground_truth_map):
  # get bounding box from mask
  y_indices, x_indices = np.where(ground_truth_map > 0)
  x_min, x_max = np.min(x_indices), np.max(x_indices)
  y_min, y_max = np.min(y_indices), np.max(y_indices)
  # add perturbation to bounding box coordinates
  H, W = ground_truth_map.shape
  x_min = max(0, x_min - np.random.randint(0, 20))
  x_max = min(W, x_max + np.random.randint(0, 20))
  y_min = max(0, y_min - np.random.randint(0, 20))
  y_max = min(H, y_max + np.random.randint(0, 20))
  bbox = [x_min, y_min, x_max, y_max]

  return bbox

In [ ]:
import matplotlib.pyplot as plt

def show_box(box, ax):
    x0, y0 = box[0], box[1]
    w, h = box[2] - box[0], box[3] - box[1]
    ax.add_patch(plt.Rectangle((x0, y0), w, h, edgecolor='green', facecolor=(0,0,0,0), lw=2))  

def show_boxes_on_image(raw_image, boxes):
    plt.figure(figsize=(10,10))
    plt.imshow(raw_image)
    for box in boxes:
      show_box(box, plt.gca())
    plt.axis('on')
    plt.show()

In [ ]:
show_boxes_on_image(img, [input_boxes])

In [ ]:
# import numpy as np
# import matplotlib.pyplot as plt
# import gc

# def show_mask(mask, ax, random_color=False):
#     if random_color:
#         color = np.concatenate([np.random.random(3), np.array([0.6])], axis=0)
#     else:
#         color = np.array([30 / 255, 144 / 255, 255 / 255, 0.6])
#     h, w = mask.shape[-2:]
#     mask_image = mask.reshape(h, w, 1) * color.reshape(1, 1, -1)
#     ax.imshow(mask_image)
#     del mask
#     gc.collect()

# def show_masks_on_image(raw_image, masks):
#   plt.imshow(np.array(raw_image))
#   ax = plt.gca()
#   ax.set_autoscale_on(False)
#   for mask in masks:
#       show_mask(mask, ax=ax, random_color=True)
#   plt.axis("off")
#   plt.show()
#   del mask
#   gc.collect()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import gc

def show_mask(mask, ax, random_color=False):
    if random_color:
        color = np.concatenate([np.random.random(3), np.array([0.6])], axis=0)
    else:
        color = np.array([30 / 255, 144 / 255, 255 / 255, 0.6])
    h, w = mask.shape[-2:]
    mask_image = mask.reshape(h, w, 1) * color.reshape(1, 1, -1)
    ax.imshow(mask_image)
    del mask
    gc.collect()

def show_masks_on_image(raw_image, masks,save_path,random_color=True):


    for mask in masks:
        if random_color:
            color = np.concatenate(
                [np.random.random(3), np.array([0.6])], axis=0)
        else:
            color = np.array([30 / 255, 144 / 255, 255 / 255, 0.6])
        h, w = mask.shape[-2:]
        mask_image = mask.reshape(h, w, 1) * color.reshape(1, 1, -1)
        plt.gca().imshow(mask_image)

    plt.axis('off')
    plt.savefig(save_path)

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from transformers import pipeline
from PIL import Image
import os


dest = "/home/harshabommana/idd20kII/sam_generated_masks"
source =  "/home/harshabommana/idd20kII/leftImg8bit/train"

def show_masks_on_image(raw_image, masks,save_path,random_color=True):


    for mask in masks:
        if random_color:
            color = np.concatenate(
                [np.random.random(3), np.array([0.6])], axis=0)
        else:
            color = np.array([30 / 255, 144 / 255, 255 / 255, 0.6])
        h, w = mask.shape[-2:]
        mask_image = mask.reshape(h, w, 1) * color.reshape(1, 1, -1)
        plt.gca().imshow(mask_image)

    plt.axis('off')
    plt.savefig(save_path)

/opt/conda/envs/sd/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
def predict(folder,file):    

    file_name = os.path.join(source,folder,file)
    print(file_name)
    img = Image.open(file_name).convert("RGB")

    generator = pipeline("mask-generation", model="facebook/sam-vit-base", device=0)
    outputs = generator(img, points_per_batch=8)
    masks = outputs["masks"]
    dest_path = os.path.join(dest,folder,file_name)
    print(dest_path)
    # show_masks_on_image(img, masks,dest_path)

In [2]:

for folder in os.listdir(source):
    folder = "201"
    folder_name = os.path.join(source,folder)
    # print(folder_name)
    images = []
    if os.path.isdir(folder_name):
        for file in os.listdir(folder_name):
            file_name = os.path.join(folder_name,file)
            file_names.append(file_name)
            img = Image.open(file_name).convert("RGB")
            images.append(img)
            # predict(folder,file)
            
    break

In [3]:
images

[<PIL.Image.Image image mode=RGB size=1920x1080>,
 <PIL.Image.Image image mode=RGB size=1920x1080>,
 <PIL.Image.Image image mode=RGB size=1920x1080>,
 <PIL.Image.Image image mode=RGB size=1920x1080>,
 <PIL.Image.Image image mode=RGB size=1920x1080>,
 <PIL.Image.Image image mode=RGB size=1920x1080>,
 <PIL.Image.Image image mode=RGB size=1920x1080>,
 <PIL.Image.Image image mode=RGB size=1920x1080>,
 <PIL.Image.Image image mode=RGB size=1920x1080>,
 <PIL.Image.Image image mode=RGB size=1920x1080>,
 <PIL.Image.Image image mode=RGB size=1920x1080>]

In [1]:
folder

NameError: name 'folder' is not defined

In [5]:
generator = pipeline("mask-generation", model="facebook/sam-vit-base", device=0)


In [6]:
outputs = generator(images, points_per_batch=8)

/opt/conda/envs/sd/lib/python3.9/site-packages/torch/utils/data/dataloader.py:642: UserWarning: Length of IterableDataset <transformers.pipelines.pt_utils.PipelineChunkIterator object at 0x7fcbc6bdf130> was reported to be 11 (when accessing len(dataloader)), but 12 samples have been fetched. 
  warnings.warn(warn_msg)
/opt/conda/envs/sd/lib/python3.9/site-packages/torch/utils/data/dataloader.py:642: UserWarning: Length of IterableDataset <transformers.pipelines.pt_utils.PipelineChunkIterator object at 0x7fcbc6bdf130> was reported to be 11 (when accessing len(dataloader)), but 13 samples have been fetched. 
  warnings.warn(warn_msg)
/opt/conda/envs/sd/lib/python3.9/site-packages/torch/utils/data/dataloader.py:642: UserWarning: Length of IterableDataset <transformers.pipelines.pt_utils.PipelineChunkIterator object at 0x7fcbc6bdf130> was reported to be 11 (when accessing len(dataloader)), but 14 samples have been fetched. 
  warnings.warn(warn_msg)
/opt/conda/envs/sd/lib/python3.9/site-pa

In [9]:
len(outputs)

11

In [11]:
for i in len(outputs):
    masks = outputs[i]["masks"]
    dest_path = os.path.join(dest,folder,file_name)

[array([[ True,  True,  True, ...,  True,  True,  True],
        [ True,  True,  True, ...,  True,  True,  True],
        [ True,  True,  True, ...,  True,  True,  True],
        ...,
        [False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False]]),
 array([[False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False],
        ...,
        [False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False]]),
 array([[False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False],
        ...,
        [False, False, False, ..., False, False, False],
        [False, False, False, ..., False, Fal

In [ ]:
images

In [ ]:
from PIL import Image

plt.imshow(img)

In [ ]:
len(masks)

In [ ]:
plt.imshow(np.array(img))
ax = plt.gca()
ax.set_autoscale_on(False)
#   for mask in masks:
show_mask(masks[64], ax=ax, random_color=True)
plt.axis("off")
plt.show()

In [ ]:
outputs = generator(img, points_per_batch=16)

In [ ]:
masks.size()

In [ ]:

show_masks_on_image(img, masks)

In [ ]:
cd /home/harshabommana/idd20kII/leftImg8bit/train

In [ ]:
import os
import shutil

In [ ]:
source_directory = "/home/harshabommana/idd20kII/leftImg8bit/train"

for folder in os.listdir(source_directory):
    folder_name = os.path.join(source_directory,folder)
    if os.path.isdir(folder_name):
        # print("folder",folder_name)
        for file in os.listdir(folder_name):
            file_name = os.path.join(folder,file)
            print("file_name",file_name)

In [ ]:
destination_directory = "/home/harshabommana/idd20kII/leftImg8bit/train"
source_directory = "/home/harshabommana/idd20kII/leftImg8bit/train"

for folder in os.listdir(source_directory):
    folder_name = os.path.join(source_directory,folder)
    if os.path.isdir(folder_name):
        # print("folder",folder_name)
        for file in os.listdir(folder_name):
            print(file)
            print(folder)
            file = file[:5]+"_"+folder+"_"+file[5:]
            # print(file)
            # break
            file_name = os.path.join(folder,file)

            print("file_name",file_name)
            if not os.path.exists(file_name):
                shutil.move(file_name,destination_directory)
  
                


In [ ]:
rm -r /home/harshabommana/idd20kII